In [1]:
%pip install psycopg2
%pip install nbformat

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\gogco\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\gogco\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import psycopg2 as pg
import pandas as pd

### Введите данные для подключения к существующей или создание новой БД

In [3]:
DB_NAME = "hw2"
DB_USER = "postgres"
DB_PASSWORD = ""
DB_HOST = "localhost"
DB_PORT = "5433"

In [4]:
# подключение к системной БД
conn = pg.connect(
    dbname="postgres",
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
conn.autocommit = True
cur = conn.cursor()

In [5]:
# попытка создания БД с указанным именем
cur.execute(f"SELECT 1 FROM pg_database WHERE datname = '{DB_NAME}';")
if not cur.fetchone():
    cur.execute(f"create database {DB_NAME} with owner = {DB_USER};")
    print(f"База данных {DB_NAME} успешно создана!")
else:
    print(f"База данных {DB_NAME} уже существует.")
cur.close()
conn.close()

База данных hw2 уже существует.


In [8]:
# подключение к существующей или созданной БД
conn = pg.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cur = conn.cursor()

### Создание таблиц и отношений БД SQL-запросом

In [ ]:
try:
    with open("schema.sql", "r", encoding="utf-8") as f:
        sql_script = f.read()
    cur.execute(sql_script)
    conn.commit()

    # проверка, создались ли таблицы
    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
    tables = cur.fetchall()

    if tables:
        print("Таблицы успешно созданы!")
    else:
        print("Ошибка: таблицы не были созданы!")

except Exception as e:
    print(f"Ошибка при создании таблиц: {e}")
    conn.rollback()  # откат изменений, если была ошибка

In [10]:
# функция изменяет разделитель в CSV-файле
def change_csv_delimiter(input_file, output_file=None, old_delimiter=';', new_delimiter=','):
    df = pd.read_csv(input_file, delimiter=old_delimiter)
    if output_file is None:
        output_file = input_file
    df.to_csv(output_file, sep=new_delimiter, index=False)
    
# функция изменяет разделитель чисел с плавающей точкой
def change_format_numbers(series: pd.Series, old_delimiter=',', new_delimiter='.'):
    return pd.to_numeric(series.astype(str).str.replace(old_delimiter, new_delimiter, regex=False), errors='coerce')

# функция формат данных в CSV-файле
def change_format_data(series: pd.Series, old_format='%d.%m.%Y', new_format='%Y-%m-%d'):
    return pd.to_datetime(series, format=old_format).dt.strftime(new_format)

In [ ]:
change_csv_delimiter('data/customer.csv')
change_csv_delimiter('data/transaction.csv')

In [ ]:
transaction_df = pd.read_csv("data/transaction.csv", delimiter=",")
transaction_df["standard_cost"] = change_format_numbers(transaction_df["standard_cost"])
transaction_df["list_price"] = change_format_numbers(transaction_df["list_price"])
transaction_df["transaction_date"] = change_format_data(transaction_df["transaction_date"])
transaction_df = transaction_df[transaction_df["customer_id"] <= 4000] # важно! выполнено удаление всех транзакций, которые имеют невалидный customer_id 
                                                                       #(вероятно, изначально таблица customer имела больше записей, 
                                                                       # а таблица transaction хранит транзакции уже несуществующих клиентов)
transaction_df.to_csv("data/transaction.csv", index = False)

In [9]:
# функция загрузки CSV в PostgreSQL
def load_csv_to_postgres(cur, csv_path, table_name):
    df = pd.read_csv(csv_path)
    # открытие файла и загрузка в PostgreSQL
    with open(csv_path, 'r', encoding='utf-8') as f:
        next(f)  # пропуск заголовков
        try:
            cur.copy_from(f, table_name, sep=',', null='')
            conn.commit()
            print(f"Данные из {csv_path} загружены в {table_name}")
        except Exception as e:
            conn.rollback()
            print(f"Ошибка при загрузке {csv_path} в {table_name}: {e}")

# функция выполняет SQL-запросы
def execute_sql_file(file_path, conn):
    try: 
        # читает SQL-файл
        with open(file_path, 'r', encoding='utf-8') as sql_file:
            sql_script = sql_file.read()
        
        # выполняет SQL-запрос
        # cur.execute(sql_script)
        # conn.commit()  # фиксирует изменения
        
        df = pd.read_sql_query(sql_script, conn)
        print(f"SQL-файл {file_path} выполнен успешно.")
        return df
    
    except Exception as e:
        conn.rollback()  # отменяет изменения при ошибке
        print(f"Ошибка при выполнении {file_path}: {e}")

In [14]:
# загрузка данных в таблицы (порядок загрузки индивидуален в соответствии со схемой)
load_csv_to_postgres(cur, "data/customer.csv", "customer")
load_csv_to_postgres(cur, "data/transaction.csv", "transaction")

Данные из data/customer.csv загружены в customer
Данные из data/transaction.csv загружены в transaction


In [10]:
execute_sql_file("sql/1.sql", conn)

SQL-файл sql/1.sql выполнен успешно.


C:\Users\gogco\AppData\Local\Temp\ipykernel_23412\3059630634.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_script, conn)


,count,job_industry_category
0,799,Manufacturing
1,774,Financial Services
2,656,None
3,602,Health
4,358,Retail
5,267,Property
6,223,IT
7,136,Entertainment
8,113,Argiculture
9,72,Telecommunications


In [11]:
pd.set_option("display.max_rows", None)  # Показывать все строки
pd.set_option("display.max_columns", None)  # Показывать все столбцы
execute_sql_file("sql/2.sql", conn)


SQL-файл sql/2.sql выполнен успешно.


C:\Users\gogco\AppData\Local\Temp\ipykernel_23412\3059630634.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_script, conn)


,month,job_industry_category,total_transactions
0,2017-01-01,Argiculture,43513.82
1,2017-01-01,Entertainment,64089.92
2,2017-01-01,Financial Services,366383.71
3,2017-01-01,Health,286860.38
4,2017-01-01,IT,107783.37
5,2017-01-01,Manufacturing,365232.45
6,2017-01-01,Property,100686.97
7,2017-01-01,Retail,182375.75
8,2017-01-01,Telecommunications,31210.20
9,2017-01-01,None,316819.75


In [12]:
execute_sql_file("sql/3.sql", conn)

SQL-файл sql/3.sql выполнен успешно.


C:\Users\gogco\AppData\Local\Temp\ipykernel_23412\3059630634.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_script, conn)


,brand,online_order_count
0,Solex,101
1,Norco Bicycles,92
2,WeareA2B,90
3,Giant Bicycles,89
4,Trek Bicycles,82
5,OHM Cycles,78
6,None,8


In [13]:
execute_sql_file("sql/4_1.sql", conn).head(100)

SQL-файл sql/4_1.sql выполнен успешно.


C:\Users\gogco\AppData\Local\Temp\ipykernel_23412\3059630634.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_script, conn)


,customer_id,total_price,max_price,min_price,txn_count
0,2183,19071.32,2005.66,230.91,14
1,1129,18349.27,1992.93,290.62,13
2,1597,18052.68,2091.47,360.40,12
3,941,17898.46,2091.47,1057.51,10
4,2788,17258.94,2083.94,183.86,11
5,936,17160.24,2005.66,183.86,12
6,1887,17133.93,2091.47,688.63,11
7,1302,17035.83,1977.36,71.16,13
8,1140,16199.24,2083.94,183.86,13
9,2309,16122.34,2091.47,290.62,12


In [16]:
execute_sql_file("sql/4_2.sql", conn).head(100)

SQL-файл sql/4_2.sql выполнен успешно.


C:\Users\gogco\AppData\Local\Temp\ipykernel_23412\3059630634.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_script, conn)


,customer_id,total_price,max_price,min_price,txn_count
0,2183,19071.32,2005.66,230.91,14
1,2183,19071.32,2005.66,230.91,14
2,2183,19071.32,2005.66,230.91,14
3,2183,19071.32,2005.66,230.91,14
4,2183,19071.32,2005.66,230.91,14
5,2183,19071.32,2005.66,230.91,14
6,2183,19071.32,2005.66,230.91,14
7,2183,19071.32,2005.66,230.91,14
8,2183,19071.32,2005.66,230.91,14
9,2183,19071.32,2005.66,230.91,14


In [37]:
execute_sql_file("sql/5_1.sql", conn)

SQL-файл sql/5_1.sql выполнен успешно.


C:\Users\gogco\AppData\Local\Temp\ipykernel_20472\3059630634.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_script, conn)


,first_name,last_name,total_spent
0,Hamlen,Slograve,60.34


In [38]:
execute_sql_file("sql/5_2.sql", conn)

SQL-файл sql/5_2.sql выполнен успешно.


C:\Users\gogco\AppData\Local\Temp\ipykernel_20472\3059630634.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_script, conn)


,first_name,last_name,total_spent
0,Jillie,Fyndon,19071.32


In [40]:
execute_sql_file("sql/6.sql", conn).head(100)

SQL-файл sql/6.sql выполнен успешно.


C:\Users\gogco\AppData\Local\Temp\ipykernel_20472\3059630634.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_script, conn)


,customer_id,transaction_id,transaction_date,list_price
0,1,9785,2017-01-05,360.40
1,2,2261,2017-05-04,1403.50
2,3,10302,2017-02-23,1311.44
3,4,12441,2017-04-03,569.56
4,5,2291,2017-03-03,688.63
5,6,7096,2017-01-28,1769.64
6,7,18369,2017-02-18,360.40
7,8,10792,2017-01-04,1362.99
8,9,8591,2017-02-04,792.90
9,10,5956,2017-06-20,1071.23


In [47]:
execute_sql_file("sql/7.sql", conn)

SQL-файл sql/7.sql выполнен успешно.


C:\Users\gogco\AppData\Local\Temp\ipykernel_20472\3059630634.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_script, conn)


,first_name,last_name,job_title,max_diff
0,Susanetta,None,Legal Assistant,357
1,Stoddard,Giacomoni,Structural Analysis Engineer,330
2,Royall,Terris,Geological Engineer,330
3,Gregorius,Cockram,Data Coordiator,330
4,Bearnard,Letixier,None,329
5,Caralie,Sellors,Senior Editor,321
6,Debee,Martynov,Senior Editor,320
7,Genni,Larway,Environmental Specialist,314
8,Franz,Craddy,None,310
9,Carolynn,Samsin,Pharmacist,310


In [27]:
cur.close()
conn.close()